## Conctere Model
- $min \ 2x_1 + 3x_2$
- S.T.
    - $3x_1 + 4x_2 \geq 1$
    - $x_1, x_2 \geq 0$

In [1]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr=2*model.x[1] + 3*model.x[2])
model.Constraint1 = pyo.Constraint(expr=3*model.x[1] + 4*model.x[2] >= 1)

In [4]:
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[1] + 3*x[2]

1 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :   1.0 : 3*x[1] + 4*x[2] :  +Inf :   True

3 Declarations: x OBJ Constraint1


## Abstract Model
- $min \ \sum_{j=1}^n \ c_jx_j$
- S.T.
    - $\sum_{j=1}^n a_ijx_j \geq b_i$ for $i = 1...m$
    - $x_j \geq 0$ for $j = 1...n$

In [1]:
import pyomo.environ as pyo

model = pyo.AbstractModel()

model.m = pyo.Param(within=pyo.NonNegativeIntegers)
model.n = pyo.Param(within=pyo.NonNegativeIntegers)

model.I = pyo.RangeSet(1, model.m)
model.J = pyo.RangeSet(1, model.n)

model.a = pyo.Param(model.I, model.J)
model.b = pyo.Param(model.I)
model.c = pyo.Param(model.J)

# The next line decclares a variable indexed by the set J
model.x = pyo.Var(model.J, domain=pyo.NonNegativeReals)

def obj_expression(m):
    return pyo.summation(m.c, m.x)

model.OBJ = pyo.Objective(rule=obj_expression)

def ax_constraint_rule(m, i):
    # return the expression for the constaint for i
    return sum(m.a[i,j] * m.x[j] for j in m.J) >= m.b[i]

# The next line creates one constraint for each member of the set model.I
model.AxbConstraint = pyo.Constraint(model.I, rule=ax_constraint_rule)